## part 8:
 Plot peak current for different voltages and determine peak late current and peak early 
current

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt

In [ ]:
def ode_euler(init, x, dif, v):
    # function is completly similar to the one introduced before except 'dif' function inputs
    output = np.zeros(len(x))
    output[0] = init
    for i in range(0, len(x) - 1):
        # In this function 'dif' get output[i] and v insted of x[i]
        output[i + 1] = output[i] + (x[i + 1] - x[i]) * dif(output[i], v)
    return output

In [ ]:
def n_prime(n, v):
    vrest = -60
    v = v - vrest
    # in v = 10 we would have a dividing by zero two following lines are written to avoide this
    if v == 10:
        v = v + 0.001
    # transsition rate from close to open state:
    alpha = 0.01 * (10 - v) / (math.exp((10 - v) / 10) - 1)

    # transsition rate from open to close state:
    beta = 0.125 * math.exp(-v / 80)

    # calculating dn from the given formula
    dn = alpha * (1 - n) - beta * n
    return dn

In [ ]:
def m_prime(m, v):
    # Na resting potential
    Ena = 53.4
    # cell resting potential
    vrest = -60
    v = v - vrest
    # in v = 25 we would have a dividing by zero two following lines are written to avoide this
    if v == 25:
        v = v + 0.001
    initial_m = 1
    # transsition rate from close to open state:
    alpha = 0.1 * (25 - v) / (math.exp(0.1 * (25 - v)) - 1)

    # transsition rate from open to close state:
    beta = 4 * math.exp(-v / 18)

    dm = alpha * (1 - m) - beta * m
    return dm

In [ ]:
def h_prime(h, v):
    # Na resting potential
    Ena = 53.4
    # cell resting potential
    vrest = -60
    v = v - vrest
    # in v = 30 we would have a dividing by zero two following lines are written to avoide this
    if v == 30:
        v = v + 0.001
    initial_h = 0
    # transsition rate from inactive to active state:
    alpha = 0.07 * math.exp(-v / 20)

    # transsition rate from active to inactive state:
    beta = 1 / (math.exp((30 - v) / 10) + 1)
    dh = (1 - h) * alpha - h * beta

    return dh

In [ ]:
def K_v(t, v):
    initial_n = 0
    gk = 36
    # calculating n from its derivative by euler function:
    n = ode_euler(initial_n, t, n_prime, v)
    # clculating potassium current i = v*g (voltage is in mV so it sould be multiplied by 0.001 )
    # cosidering the probebilty of channels being open
    ik = 0.001 * (v + 72.1) * n ** 4 * gk
    return ik

In [ ]:
def Na_v(t, v):
    initial_m = 0
    initial_h = 1

    Ena = 53
    gna = 120

    # calculating m and h from its derivative by euler function:
    m = ode_euler(initial_m, t, m_prime, v)
    h = ode_euler(initial_h, t, h_prime, v)

    # clculating sodium current i = v*g (voltage is in mV so it sould be multiplied by 0.001 )
    # considering the probebilty of channels being open:
    ina = 0.001 * h * (m ** 3) * gna * (v - Ena)
    return ina

In [ ]:
t = np.arange(0, 20, 0.1)
v = np.arange(-80, 80, 1)
l = len(v)
v_max = np.zeros(l)
for i in range(0, l - 1):
    v_max[i] = max(K_v(t, v[i]))
plt.figure()
plt.plot(v, v_max)
for i in range(0, l - 1):
    #since Na current sighns change by increasing the voltage we shuold cosider it by changing min to max after v = 53
    if v[i] <= 53:
        v_max[i] = min(Na_v(t, v[i]))
    else:
        v_max[i] = max(Na_v(t, v[i]))

plt.plot(v, v_max)